In [1]:
import glob
import datetime
import time
import os

In [ ]:
# Setting up path names -- you will need to adapt this to your system

ifg_data_folder = '/uufs/chpc.utah.edu/common/home/lin-group15/agm/em27/ua/inst_data/ifgs' #where the ifg date folders are
date = '20230516' #the date of interest -- I think this is the only one with bad ifgs for the TCCON side by side
date_folder = os.path.join(ifg_data_folder,date)

In [ ]:
#Find which ifgs are good and which are empty
good_data_ifgs = [] #ifgs with stuff in them
no_data_ifgs = [] #empty ifgs

for file in sorted(os.listdir(date_folder)): #loop through the data folder
    fsize_kb = os.stat(os.path.join(date_folder,file)).st_size/1024 #get the size of the file
    if fsize_kb>2: #if the filesize is bigger than 2kb
        good_data_ifgs.append(file) #add it to the good data list
    else:
        no_data_ifgs.append(file) #if not, add it to the no data list
print(f'IFGS with good data: {good_data_ifgs}')
print(f'IFGS with no data: {no_data_ifgs}')

In [ ]:
# BEFORE YOU RUN THIS CELL
# Make sure you have your data backed up as it will delete IFGS identified as having no data. 
# You will only have to run this once, then the files will be gone so you obviously cant "delete them again"
for file in no_data_ifgs:
    os.remove(os.path.join(date_folder,file))

In [ ]:
# Now we should have a folder with a gap in IFGS -- a set of good ifgs before xxx.3600 and a set of good
# ifgs after xxx.3600. We want to rename them so that we have a continuous set of data starting at xxx.0001

before_3600_ids = [] #all of the ifgs with an id before xxx.3600
after_3600_ids = [] #all of the ifgs with an id after xxx.3600
for file in sorted(os.listdir(date_folder)): #loop through the data folder
    ifg_id =int(file.split('.')[-1]) #get the ifg id by stripping off everything after the last "."
    if ifg_id < 3601: #if it's before 3601
        before_3600_ids.append(ifg_id) #add it to the before list
    else:
        after_3600_ids.append(ifg_id) #otherwise add it to the after list
last_before_3600_id = max(before_3600_ids) #this is the id of the last good ifg before 3600

In [ ]:
# AGAIN: you should only run this cell once and it will rename files in your directory, so make sure you know
# where you're working

new_id = last_before_3600_id #initialize the first "new_id" as the last good ifg before 3600
for old_id in after_3600_ids: #loop through all of the after 3600 ifgs, and rename them to be sequential starting with the last id before 3600
    new_id = new_id+1 #add one to the new_id, which will increment every time
    new_id_str = f'{new_id:04}' #make it a string of 4 digits with leading 0s 

    old_fullname = os.path.join(date_folder,f'ua{date}.ifg.{old_id}') #this is the old filename and path of the good ifg
    new_fullname = os.path.join(date_folder,f'ua{date}.ifg.{new_id_str}') #this is the new filename incremented one more than the previous
    os.rename(old_fullname,new_fullname) #this actually renames the file from the old to the new name